6.3 Семинар: cлой нормализации
4 из 11 шагов пройдено
2 из 8 баллов  получено

В этом уроке мы детально изучим слои нормализации.

Самая популярная версия слоя нормализации - слой нормализации "по батчу" (batch-norm слой).


В данном шаге вам требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

Параметр Бета принимается равным 0.
Параметр Гамма принимается равным 1.
Функция должна корректно работать только на этапе обучения.
Вход имеет размерность число элементов в батче * длина каждого инстанса.
 
Очень внимательно посмотрите на определение функции, вычисляющей std.

Sample Input: anything

Sample Output: True

In [1]:
import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    mean = input_tensor.mean(dim=0)
    var = input_tensor.var(dim=0, unbiased=False)
    
    normed_tensor = (input_tensor - mean) / torch.sqrt(var + eps)
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# import numpy as np
# all_correct = True
# for eps_power in range(10):
#     eps = np.power(10., -eps_power)
#     batch_norm.eps = eps
#     batch_norm_out = batch_norm(input_tensor)
#     custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

#     all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
#     all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
# print(all_correct)

6.3 Семинар: cлой нормализации
4 из 11 шагов пройдено
2 из 8 баллов  получено

Немного обобщим функцию с предыдущего шага - добавим возможность задавать параметры Бета и Гамма.

На данном шаге вам требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

Функция должна корректно работать только на этапе обучения.
Вход имеет размерность число элементов в батче * длина каждого инстанса.

Sample Input: anything

Sample Output: True

In [2]:
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    # Шаг 1: Вычисление среднего значения
    mean = input_tensor.mean(dim=0)
    
    # Шаг 2: Вычетание среднего значения
    centered_input = input_tensor - mean
    
    # Шаг 3: Вычисление дисперсии и нормализация
    variance = centered_input.pow(2).mean(dim=0)
    stddev = torch.sqrt(variance + eps)
    normalized_input = centered_input / stddev
    
    # Шаги 4 и 5: Применение весового коэффициента и смещения
    normed_tensor = weight * normalized_input + bias
    
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# batch_norm = nn.BatchNorm1d(input_size, eps=eps)
# batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
# batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
# batch_norm_out = batch_norm(input_tensor)
# custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
# print(torch.allclose(batch_norm_out, custom_batch_norm_out, 1e-3) \
#       and batch_norm_out.shape == custom_batch_norm_out.shape)

6.3 Семинар: cлой нормализации
4 из 11 шагов пройдено
2 из 8 баллов  получено

Избавимся еще от одного упрощения - реализуем работу слоя батч-нормализации на этапе предсказания.

На этом этапе вместо статистик по батчу будем использовать экспоненциально сглаженные статистики из истории обучения слоя.

В данном шаге вам требуется реализовать полноценный класс батч-нормализации без использования стандартной функции, принимающий на вход двумерный тензор. Осторожно, расчёт дисперсии ведётся по смещенной выборке, а расчет скользящего среднего по несмещенной.

Sample Input: anything

Sample Output: True

```python
import torch
import torch.nn as nn


input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        # Реализуйте в этом месте конструктор.

    def __call__(self, input_tensor):
        normed_tensor = # Напишите в этом месте нормирование входного тензора.

    def eval(self):
        # В этом методе реализуйте переключение в режим предикта.


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# all_correct = True

# for i in range(8):
#     torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
#     norm_output = batch_norm(torch_input)
#     custom_output = custom_batch_norm1d(torch_input)
#     all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
#         and norm_output.shape == custom_output.shape

# batch_norm.eval()
# custom_batch_norm1d.eval()

# for i in range(8):
#     torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
#     norm_output = batch_norm(torch_input)
#     custom_output = custom_batch_norm1d(torch_input)
#     all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
#         and norm_output.shape == custom_output.shape
# print(all_correct)
```

In [15]:
import torch
import torch.nn as nn

class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.running_mean = torch.zeros(weight.shape[0])
        self.running_var = torch.ones(weight.shape[0])
        self.is_training = True

    def __call__(self, input_tensor):
        if self.is_training:
            # Расчёт текущих средних и дисперсий
            batch_mean = input_tensor.mean(dim=0)
            batch_var = ((input_tensor - batch_mean) ** 2).mean(dim=0)

            # Обновление экспоненциальных средних
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var
            # (1-momentum)*var*batch_size/(batch_size-1)+momentum*running_var

            # Смещённая оценка дисперсии
            biased_var = batch_var * (input_tensor.size(0) / (input_tensor.size(0) - 1))

            # Нормализация
            normed_tensor = (input_tensor - batch_mean) / torch.sqrt(biased_var + self.eps)
        else:
            # Использование экспоненциально сглаженных средних и дисперсий
            normed_tensor = (input_tensor - self.running_mean) / torch.sqrt(self.running_var + self.eps)

        # Применение весов и смещений
        normed_tensor = self.weight * normed_tensor + self.bias

        return normed_tensor

    def eval(self):
        self.is_training = False

# Настройки
input_size = 3
batch_size = 5
eps = 1e-1

# Стандартный слой батч-нормализации
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

# Пользовательский слой батч-нормализации
custom_batch_norm1d = CustomBatchNorm1d(
    batch_norm.weight.data,
    batch_norm.bias.data,
    eps,
    batch_norm.momentum
)



In [19]:

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape
print(all_correct)

print("custom_output\n", custom_output)
print("norm_output\n", norm_output)

False
custom_output
 tensor([[ 0.4575,  0.5046, -0.1942],
        [ 0.2927, -2.6247, -0.0287],
        [-0.1823,  1.8898, -0.2639],
        [ 0.3403,  1.4986, -0.4484],
        [-0.4431, -0.1873, -0.4977]])
norm_output
 tensor([[ 0.4066,  0.4778, -0.1743],
        [ 0.2574, -2.3622, -0.0239],
        [-0.1724,  1.7349, -0.2377],
        [ 0.3005,  1.3799, -0.4053],
        [-0.4084, -0.1502, -0.4501]], grad_fn=<NativeBatchNormBackward0>)
